In [1]:
import torch
torch.__version__

'1.7.1+cu110'

In [3]:
import csv
import pandas as pd
# 读取CSV文件并将数据存储在pandas DataFrame中
df = pd.read_csv('/mnt/mydisk/beijing.csv')
# 将前一半行写入第一个CSV文件,作为 train 数据
df[:201].to_csv('data1.csv', index=False)
# 将后一半行写入第二个CSV文件,作为 validation 数据
df[201:].to_csv('data2.csv', index=False)

# 测试GPU是否可用

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)
from transformers import BertModel
#加载预训练模型
pretrained = BertModel.from_pretrained('/mnt/mydisk/bert-base-chinese')
#需要移动到cuda上
pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#定义下游任务模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()
#同样要移动到cuda
model.to(device)

#虚拟一批数据,需要把所有的数据都移动到cuda上
input_ids = torch.ones(16, 100).long().to(device)
attention_mask = torch.ones(16, 100).long().to(device)
token_type_ids = torch.ones(16, 100).long().to(device)
labels = torch.ones(16).long().to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

#后面的计算和中文分类完全一样，只是放在了cuda上计算

device= cuda


Some weights of the model checkpoint at /mnt/mydisk/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 2])

# Start


In [5]:
import pandas as pd
import torch.utils.data as data

class CSVDataset(data.Dataset):
    def __init__(self, csv_file_path, text_column, label_column):
        self.data = pd.read_csv(csv_file_path)
        self.text_column = text_column
        self.label_column = label_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx][self.text_column]
        label = self.data.iloc[idx][self.label_column]
        return text, label
    

dataset = CSVDataset('/mnt/mydisk/CSTD/data1.csv', 'text', 'label')
len(dataset)
for i in dataset:
    print(i)

('当前，全市学习贯彻习近平新时代中国特色社会主义思想主题教育正深入开展。5月8日上午，市委书记尹力围绕“为什么学”“学什么”“怎么学”“怎么干”等四个方面，在市委党校为党员干部讲了一堂主题教育专题党课。他强调，要牢牢把握“学思想、强党性、重实践、建新功”的总要求，集中精力抓好理论学习、调查研究、推动发展、检视整改，推动习近平新时代中国特色社会主义思想在京华大地落地生根，形成生动实践，以奋发有为的精神状态和真抓实干的实际行动，努力创造经得起历史和人民检验的实绩。学习贯彻习近平新时代中国特色社会主义思想主题教育中央第一指导组组长车俊，市委副书记、市长殷勇，市人大常委会主任李秀领，市政协主席魏小东，市委副书记刘伟，中央第一指导组副组长李小三参加。从治国理政新理念新思想新战略，到“十个明确”“十四个坚持”“十三个方面成就”，再到构建人类命运共同体等一系列中国主张，尹力引导大家深入思考“为什么学”。他指出，习近平新时代中国特色社会主义思想实现了马克思主义中国化时代化新的飞跃，我们要在“为什么学”上提高政治站位，持续增强学习的思想自觉和行动自觉。习近平新时代中国特色社会主义思想深刻回答了一系列重大时代课题，实现了对中国特色社会主义建设规律认识的新跃升；坚持以“两个结合”推进理论创新，标注了马克思主义发展的新高度；指引新时代实践取得历史性成就，展现出强大的真理力量和实践伟力；为全世界提供了更多中国智慧和中国方案，引领我国日益走近世界舞台中央。有这一思想的科学指引，我们就有了理论灯塔、思想旗帜、行动指南，就能够始终坚持正确方向，不断把党和人民事业发展推向新高度。“风雨不动安如山”般的理想信念、“江山就是人民、人民就是江山”的为民情怀……尹力向大家详细阐述了“学什么”。他指出，习近平新时代中国特色社会主义思想是博大精深、内涵丰富的科学理论体系，我们要在“学什么”上做到全面系统，真正掌握精髓要义。要细学深思坚定不移的理想信念。习近平新时代中国特色社会主义思想之所以感召人、凝聚人，首先就在于其贯穿着浓烈的信仰追求，我们要深刻把握，坚定理想信念，永葆共产党人政治本色。要细学深思真挚深厚的为民情怀。坚持以人民为中心，是习近平新时代中国特色社会主义思想的根本政治立场，我们要自觉向习近平总书记看齐，始终把人民放在心中最高位置，牢牢站稳人民立场，以实际行动换来群众更好的生活。要细学深思守正创新的

In [6]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')


def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=32,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

5


(torch.Size([32, 500]),
 torch.Size([32, 500]),
 torch.Size([32, 500]),
 tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0], device='cuda:0'))

In [8]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 200:
        break

0 0.6863791346549988 0.5625


In [9]:
model.eval()
correct = 0
total = 0

dataset2 = CSVDataset('/mnt/mydisk/CSTD/data2.csv', 'text', 'label')

loader_test = torch.utils.data.DataLoader(dataset=dataset2,
                                            batch_size=32,
                                            collate_fn=collate_fn,
                                            shuffle=True,
                                            drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):
        print(input_ids, attention_mask, token_type_ids,labels)

tensor([[ 101,  126, 3299,  ..., 7392, 4895,  102],
        [ 101,  128, 3299,  ..., 3175, 7481,  102],
        [ 101,  464, 8159,  ...,    0,    0,    0],
        ...,
        [ 101,  125, 3299,  ..., 7344, 4554,  102],
        [ 101,  464, 8158,  ...,    0,    0,    0],
        [ 101, 1333, 3403,  ..., 2428,  511,  102]], device='cuda:0') tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0') tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0') tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 0], device='cuda:0')
tensor([[ 101,  126, 3299,  ..., 3868, 4554,  102],
     

In [11]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset2,
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 10:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

0
1
2
3
4
5
6
7
0.49609375
